# Training a machine learning classifier on resting state fMRI or structural MRI data

The goal is to train a classifier on MRI data. Two suitable data sets were identified.

## Getting the data

The ABIDE data set can be downloaded using nilearn. The data set contains resting state fMRI data from multiple sites. Some of the participants had Autism Spectrum Disorder (ASD). This dataset could be used to train a classifier to distinguish ASD and control participants based on the resting state data.

The data were preprocessed with different pipelines. We will download the data preprocessed with the CPAC pipeline. For more on the differences between pipelines see [here](http://preprocessed-connectomes-project.org/abide/Pipelines.html).

We are setting quality_checked to "True", to only include subjects that passed quality control.

In [1]:
from nilearn import datasets
#CHANGE PATH TO WERE YOU WANT TO STORE THE DATA
abide = datasets.fetch_abide_pcp(data_dir="/mnt/c/brainhackschool/project/data",
                                n_subjects=2,
                                pipeline="cpac",
                                quality_checked=True,
                                )

/home/mikkel/miniconda3/lib/python3.7/site-packages/numpy/lib/npyio.py:2372: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


Another suitable dataset is [this one](https://openneuro.org/datasets/ds002726/versions/1.0.1). It contains T1 weighted structural MRI images of gifted and control children. A classifier could be trained to predict if a child is gifted or not based on cortical thickness of 308 regions.

The data can be downloaded using [AWS CLI](https://aws.amazon.com/de/cli/) or [datalad](https://www.datalad.org/). See the [download page](https://openneuro.org/datasets/ds002726/versions/1.0.1/download) of the data set for more information.